### Problems Definition

### Option Types and Pricing Techniques

Use the expected value of the discounted payoff under the risk-neutral density Q,

$$V(S, t)=e^{-r(T-t)} E^\mathbb{Q}\left[P\text{ayoff}\left(S_{T}\right)\right]$$

for the form of payoff of Asian options and lookback options. The payoff formula is listed in the following table.

| Option type | Price formula | Notation |
| --- | --- | --- |
| Asian Call option, fixed strike price | $$\text{Asian}(S,T)=\exp(-r(T-t)) \text{max}(A-E,0)$$ | A is the arithmetic average of underlying stock price from simulation,$A = \frac{1}{n} \sum_{i=1}^{n} S(t_i)$; E is the strike price, predetermined |
| Asian Call option, floating strike price | $$\text{Asian}(S,T)=\exp(-r(T-t)) \text{max}(A(T)-E, 0)$$  | A(T) is the underlying stock price at the expiry; E is the floating strike price, calculated by the average price from simulation |
| Lookback Call, maximum, fixed strike price | $$M(S, T)=\exp(-r(T-t)) \text{max(M - E,0)}$$ | M is the realized maximum, $M=\max(S(t_{i})), 1\leq i\leq n;$ E is the strike price, predetermined |
| Lookback Put, minimum, fixed strike price | $$m(\text{S, T})=\exp(-r(T-t)) \text{max(E - m,0)}$$ | m is the realized minimum, $m=\min(S(t_{i})),1\leq i\leq n;$ E is the strike price, predetermined |
| Lookback Call, minimum, floating strike price | $$ m(S,T) = \exp(-r(T-t)) \text{max(S(T) - m, 0)} $$ | S(T) is the underlying stock price at the expiry;  m is the floating strike price, $ m = \min(S(t_{i})), 1 \leq i \leq n $ |
| Lookback Put, maximum, floating strike price | $$ M(S,T) = \exp(-r(T-t)) \text{max(M - S(T), 0)} $$ |  S(T)  is the underlying stock price at the expiry; M is the floating strike price, $ M = \max(S(t_{i})), 1 \leq i \leq n $ |

**Notes:** for Asian options, the difference between calls and puts is simple from pricing point of view, but the strike/rate distinction (floating or fixed strikes) can make a big difference (Paul Wilmott, 2007). Therefore, here I just focus on call options.

### Simulating Prices

For simulating paths of the underlying stock price, use the Euler-Maruyama scheme, the formula is as follow:

$$S_{t+\Delta t} = S_t (1 + r\Delta t + \sigma\sqrt{\Delta t}W_t)$$

$$W_t \sim N(0,1) $$

With an error of $O(\sqrt{\Delta t})$

### The Initial Conditions

- Current stock price  $S _{0} = 100$ 
- Strike price  $ E = 100  $
- Time to expiry  $ (T-t) = 1 $ year
- Volatility $ \sigma = 20\%$
- Constant risk-free interest rate  $r = 5\%$
  

### Scenarios

Generating potential price paths of the underlying assets and calculating option prices based on initial conditions.

Setting scenarios for underlying price, time to expiry, volatility, and interest rate.

Discuss how these changes affect option prices.

Scenarios are as follows:

| Scenarios | Greeks |
| --- | --- |
| Underlying stock price increases by 1 | $$Delta \  = \frac{\partial V}{\partial S} $$ |
| Time to expiry decreases by 1 day | $$Theta \  = \frac{\partial V}{\partial t}$$ |
| Volatility increases by 1% | $$Vega \ = \frac{\partial V}{\partial \sigma}$$ |
| Interest rate increases by 1% | $$Rho \ = \frac{\partial V}{\partial r}$$ |

In [8]:
import warnings
warnings.filterwarnings('ignore')

from pydantic import BaseModel, Field, ValidationError, validator, computed_field
from typing import List, Tuple
import pandas as pd
import numpy as np
from scipy.stats import norm
from typing import ClassVar


from tabulate import tabulate

In [26]:
class MonteCarloOptionPricing(BaseModel):
   

    S0: float = Field(..., gt=0, description="Current stock price")
    E: float = Field(..., gt=0, alias="strike", description="Strike price")
    r: float = Field(..., ge=0, le=1, alias="rate", description="Risk-free rate")
    sigma: float = Field(..., gt=0, le=1, description="Volatility")
    T: float = Field(..., gt=0, le=3650, alias="dte", description="Days to expiration")
    N: int = Field(..., gt=0, alias="nsim", description="Number of simulations")
    Nots: int = Field(252, gt=0, alias="timesteps", description="Number of timesteps")

    class Config:
        allow_population_by_field_name = True
 
    @property
    def disc(self) -> float:
        return np.exp(-self.r * self.T)

    @property
    def randomNumber(self) -> np.ndarray:
        return np.random.standard_normal(self.N)

    @property
    def simulatePath(self) -> np.ndarray:
        np.random.seed(2024)

        dt = self.T / self.Nots

        S = np.zeros((self.Nots, self.N))
        S[0] = self.S0

        for i in range(0, self.Nots-1):
            w = self.randomNumber
            S[i+1] = S[i] * (1 + self.r*dt + self.sigma*np.sqrt(dt)*w)

        return S

    @property
    def asianOption(self) -> List[float]:
             
        S = self.simulatePath
    
        A = S.mean(axis=0)
    
        asianFix = self.disc * np.mean(np.maximum(A - self.E, 0))
        asianFloat = self.disc * np.mean(np.maximum(S[-1] - A, 0))
    
        return [asianFix, asianFloat]

    @property
    def lookbackOption(self) -> List[float]:
        
        S = self.simulatePath
        
        maxValue = S.max(axis=0)
        minValue = S.min(axis=0)

        lookbackCall = self.disc * np.mean(np.maximum(maxValue - self.E,0))
        lookbackPut = self.disc * np.mean(np.maximum(self.E - minValue,0))
        lookbackCallFl = self.disc * np.mean(np.maximum(S[-1] - minValue,0))
        lookbackPutFl = self.disc * np.mean(np.maximum(maxValue - S[-1],0))

        return [lookbackCall, lookbackPut,lookbackCallFl, lookbackPutFl]


    @property
    def EuroOption(self) -> List[float]:
        
        S = self.simulatePath

        EuroCall = self.disc * np.mean(np.maximum(S[-1] - self.E, 0))
        EuroPut = self.disc * np.mean(np.maximum(self.E - S[-1],0))

        return [EuroCall, EuroPut]

   

In [28]:
try:
    option1 = MonteCarloOptionPricing(
            S0=100,
            strike=100,
            rate=0.05,
            sigma=0.2,
            dte=1,
            nsim=100000,
            timesteps=252
        )
except ValidationError as e:
    print(f"\nValidation error: {e}")

In [243]:
print(f"Asian Option with fixed strike price: {option1.asianOption[0]:0.6f}")
print(f"Asian Option with floating strike price: {option1.asianOption[1]:0.6f}")

Asian Option with fixed strike price: 5.725051
Asian Option with floating strike price: 5.871493


In [245]:
print(f"Lookback call Option: {option1.lookbackOption[0]:0.6f}")
print(f"Lookback put Option: {option1.lookbackOption[1]:0.6f}")

Lookback call Option: 18.290788
Lookback put Option: 11.740252


In [23]:
print(f"Lookback call Option, floating strike price: {option1.lookbackOption[2]:0.6f}")
print(f"Lookback put Option, floating strike price: {option1.lookbackOption[3]:0.6f}")

Lookback call Option, floating strike price: 16.644977
Lookback put Option, floating strike price: 13.386062


In [60]:
Afix00 = round(option1.asianOption[0],6)
Afloat00 = round(option1.asianOption[1],6)
Lbc00 = round(option1.lookbackOption[0],6)
Lbp00 = round(option1.lookbackOption[1],6)
LbcFl00 = round(option1.lookbackOption[2],6)
LbpFl00 = round(option1.lookbackOption[3],6)

In [34]:
try:
    optionS1 = MonteCarloOptionPricing(
            S0=101,
            strike=100,
            rate=0.05,
            sigma=0.2,
            dte=1,
            nsim=100000,
            timesteps=252
        )
except ValidationError as e:
    print(f"\nValidation error: {e}")

In [249]:
print(f"Asian Option with fixed strike price: {optionS1.asianOption[0]:0.6f}")
print(f"Asian Option with floating strike price: {optionS1.asianOption[1]:0.6f}")

Asian Option with fixed strike price: 6.328355
Asian Option with floating strike price: 5.930208


In [251]:
print(f"Lookback call Option: {optionS1.lookbackOption[0]:0.6f}")
print(f"Lookback put Option: {optionS1.lookbackOption[1]:0.6f}")

Lookback call Option: 19.424925
Lookback put Option: 10.964583


In [29]:
print(f"Lookback call Option, floating strike price: {optionS1.lookbackOption[2]:0.6f}")
print(f"Lookback put Option, floating strike price: {optionS1.lookbackOption[3]:0.6f}")

Lookback call Option, floating strike price: 16.811427
Lookback put Option, floating strike price: 13.519923


In [62]:
Afix01 = round(optionS1.asianOption[0],6)
Afloat01 = round(optionS1.asianOption[1],6)
Lbc01 = round(optionS1.lookbackOption[0],6)
Lbp01 = round(optionS1.lookbackOption[1],6)
LbcFl01 = round(optionS1.lookbackOption[2],6)
LbpFl01 = round(optionS1.lookbackOption[3],6)

In [38]:
try:
    optionS2 = MonteCarloOptionPricing(
            S0=100,
            strike=100,
            rate=0.05,
            sigma=0.2,
            dte=1-(1/252),
            nsim=100000,
            timesteps=252
        )
except ValidationError as e:
    print(f"\nValidation error: {e}")

In [255]:
print(f"Asian Option with fixed strike price: {optionS2.asianOption[0]:0.6f}")
print(f"Asian Option with floating strike price: {optionS2.asianOption[1]:0.6f}")

Asian Option with fixed strike price: 5.711668
Asian Option with floating strike price: 5.857381


In [257]:
print(f"Lookback call Option: {optionS2.lookbackOption[0]:0.6f}")
print(f"Lookback put Option: {optionS2.lookbackOption[1]:0.6f}")

Lookback call Option: 18.249038
Lookback put Option: 11.723892


In [35]:
print(f"Lookback call Option, floating strike price: {optionS2.lookbackOption[2]:0.6f}")
print(f"Lookback put Option, floating strike price: {optionS2.lookbackOption[3]:0.6f}")

Lookback call Option, floating strike price: 16.609742
Lookback put Option, floating strike price: 13.363187


In [64]:
Afix02 = round(optionS2.asianOption[0],6)
Afloat02 = round(optionS2.asianOption[1],6)
Lbc02 = round(optionS2.lookbackOption[0],6)
Lbp02 = round(optionS2.lookbackOption[1],6)
LbcFl02 = round(optionS2.lookbackOption[2],6)
LbpFl02 = round(optionS2.lookbackOption[3],6)

In [42]:
try:
    optionS3 = MonteCarloOptionPricing(
            S0=100,
            strike=100,
            rate=0.05,
            sigma=0.2*(1 + 0.01),
            dte=1,
            nsim=100000,
            timesteps=252
        )
except ValidationError as e:
    print(f"\nValidation error: {e}")

In [261]:
print(f"Asian Option with fixed strike price: {optionS3.asianOption[0]:0.6f}")
print(f"Asian Option with floating strike price: {optionS3.asianOption[1]:0.6f}")

Asian Option with fixed strike price: 5.768164
Asian Option with floating strike price: 5.915609


In [263]:
print(f"Lookback call Option: {optionS3.lookbackOption[0]:0.6f}")
print(f"Lookback put Option: {optionS3.lookbackOption[1]:0.6f}")

Lookback call Option: 18.455225
Lookback put Option: 11.870397


In [41]:
print(f"Lookback call Option, floating strike price: {optionS3.lookbackOption[2]:0.6f}")
print(f"Lookback put Option, floating strike price: {optionS3.lookbackOption[3]:0.6f}")

Lookback call Option, floating strike price: 16.775536
Lookback put Option, floating strike price: 13.550086


In [66]:
Afix03 = round(optionS3.asianOption[0],6)
Afloat03 = round(optionS3.asianOption[1],6)
Lbc03 = round(optionS3.lookbackOption[0],6)
Lbp03 = round(optionS3.lookbackOption[1],6)
LbcFl03 = round(optionS3.lookbackOption[2],6)
LbpFl03 = round(optionS3.lookbackOption[3],6)

In [46]:
try:
    optionS4 = MonteCarloOptionPricing(
            S0=100,
            strike=100,
            rate=0.05*(1+0.01),
            sigma=0.2,
            dte=1,
            nsim=100000,
            timesteps=252
        )
except ValidationError as e:
    print(f"\nValidation error: {e}")

In [267]:
print(f"Asian Option with fixed strike price: {optionS4.asianOption[0]:0.6f}")
print(f"Asian Option with floating strike price: {optionS4.asianOption[1]:0.6f}")

Asian Option with fixed strike price: 5.737204
Asian Option with floating strike price: 5.884987


In [269]:
print(f"Lookback call Option: {optionS4.lookbackOption[0]:0.6f}")
print(f"Lookback put Option: {optionS4.lookbackOption[1]:0.6f}")

Lookback call Option: 18.313758
Lookback put Option: 11.716322


In [47]:
print(f"Lookback call Option, floating strike price: {optionS4.lookbackOption[2]:0.6f}")
print(f"Lookback put Option, floating strike price: {optionS4.lookbackOption[3]:0.6f}")

Lookback call Option, floating strike price: 16.668389
Lookback put Option, floating strike price: 13.361691


In [68]:
Afix04 = round(optionS4.asianOption[0],6)
Afloat04 = round(optionS4.asianOption[1],6)
Lbc04 = round(optionS4.lookbackOption[0],6)
Lbp04 = round(optionS4.lookbackOption[1],6)
LbcFl04 = round(optionS4.lookbackOption[2],6)
LbpFl04 = round(optionS4.lookbackOption[3],6)

In [57]:
print(Afix00, Afix01, Afix02, Afix03, Afix04)
print(Afloat00, Afloat01, Afloat02, Afloat03, Afloat04)
print(Lbc00, Lbc01, Lbc02, Lbc03, Lbc04)
print(Lbp00, Lbp01, Lbp02, Lbp03, Lbp04)
print(LbcFl00, LbcFl01, LbcFl02, LbcFl03, LbcFl04)
print(LbpFl00, LbpFl01, LbpFl02, LbpFl03, LbpFl04)

5.725051 6.328355 5.711668 5.768164 5.737204
5.871493 5.930208 5.857381 5.915609 5.884987
18.290788 19.424925 18.249038 18.455225 18.313758
11.740252 10.964583 11.723892 11.870397 11.716322
16.644977 16.811427 16.609742 16.775536 16.668389
13.386062 13.519923 13.363187 13.550086 13.361691


In [96]:
class Greeks(BaseModel):

    Afix: np.ndarray = Field(default_factory=lambda: np.zeros(5))
    Afloat: np.ndarray = Field(default_factory=lambda: np.zeros(5)) 
    Lbc: np.ndarray = Field(default_factory=lambda: np.zeros(5))
    Lbp: np.ndarray = Field(default_factory=lambda: np.zeros(5))
    LbcFl: np.ndarray = Field(default_factory=lambda: np.zeros(5))
    LbpFl: np.ndarray = Field(default_factory=lambda: np.zeros(5))

    Euroc: np.ndarray = Field(default_factory=lambda: np.zeros(5))
    Europ: np.ndarray = Field(default_factory=lambda: np.zeros(5))

    

    model_config = {"arbitrary_types_allowed": True}            

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
   
        self.Afix = np.array([Afix00, Afix01, Afix02, Afix03, Afix04])
        self.Afloat = np.array([Afloat00, Afloat01, Afloat02, Afloat03, Afloat04])
        self.Lbc = np.array([Lbc00, Lbc01, Lbc02, Lbc03, Lbc04])
        self.Lbp = np.array([Lbp00, Lbp01, Lbp02, Lbp03, Lbp04])
        self.LbcFl = np.array([LbcFl00, LbcFl01, LbcFl02, LbcFl03, LbcFl04])
        self.LbpFl = np.array([LbpFl00, LbpFl01, LbpFl02, LbpFl03, LbpFl04])

        self.Euroc = np.array([Euroc00 ,Euroc01 ,Euroc02 ,Euroc03 ,Euroc04])
        self.Europ = np.array([Europ00 ,Europ01 ,Europ02 ,Europ03 ,Europ04])
               

                            
     
    def delta(self) -> list[float]:

        AfixDelta = (self.Afix[1] - self.Afix[0])/1
        AfloatDelta = (self.Afloat[1] - self.Afloat[0])/1
        LbcDelta = (self.Lbc[1] - self.Lbc[0])/1
        LbpDelta = (self.Lbp[1] - self.Lbp[0])/1
        LbcFlDelta = (self.LbcFl[1] - self.LbcFl[0])/1
        LbpFlDelta = (self.LbpFl[1] - self.LbpFl[0])/1

        EurocDelta = (self.Euroc[1] - self.Euroc[0])/1
        EuropDelta = (self.Europ[1] - self.Europ[0])/1
        
        return [AfixDelta, AfloatDelta, LbcDelta, LbpDelta, LbcFlDelta, LbpFlDelta, EurocDelta, EuropDelta]
        

 
    def theta(self) -> list[float]:
        
        AfixTheta = (self.Afix[2] - self.Afix[0])/(1/252)
        AfloatTheta = (self.Afloat[2] - self.Afloat[0])/(1/252)
        LbcTheta = (self.Lbc[2] - self.Lbc[0])/(1/252)
        LbpTheta = (self.Lbp[2] - self.Lbp[0])/(1/252)
        LbcFlTheta = (self.LbcFl[2] - self.LbcFl[0])/(1/252)
        LbpFlTheta = (self.LbpFl[2] - self.LbpFl[0])/(1/252)

        EurocTheta = (self.Euroc[2] - self.Euroc[0])/(1/252)
        EuropTheta = (self.Europ[2] - self.Europ[0])/(1/252)
        
        return [AfixTheta, AfloatTheta, LbcTheta, LbpTheta, LbcFlTheta, LbpFlTheta, EurocTheta, EuropTheta]

    
   
    def vega(self) -> list[float]:

        AfixVega = (self.Afix[3] - self.Afix[0])/ (0.2*0.01)
        AfloatVega = (self.Afloat[3] - self.Afloat[0])/(0.2*0.01)
        LbcVega = (self.Lbc[3] - self.Lbc[0])/(0.2*0.01)
        LbpVega = (self.Lbp[3] - self.Lbp[0])/(0.2*0.01)
        LbcFlVega = (self.LbcFl[3] - self.LbcFl[0])/(0.2*0.01)
        LbpFlVega = (self.LbpFl[3] - self.LbpFl[0])/(0.2*0.01)

        EurocVega = (self.Euroc[3] - self.Euroc[0])/(0.2*0.01)
        EuropVega = (self.Europ[3] - self.Europ[0])/(0.2*0.01)
        
        return [AfixVega, AfloatVega, LbcVega, LbpVega, LbcFlVega, LbpFlVega, EurocVega, EuropVega]

  
    def rho(self) -> list[float]:

        AfixRho = (self.Afix[4] - self.Afix[0])/ (0.05*0.01)
        AfloatRho = (self.Afloat[4] - self.Afloat[0])/(0.05*0.01)
        LbcRho = (self.Lbc[4] - self.Lbc[0])/(0.05*0.01)
        LbpRho = (self.Lbp[4] - self.Lbp[0])/(0.05*0.01)
        LbcFlRho = (self.LbcFl[4] - self.LbcFl[0])/(0.05*0.01)
        LbpFlRho = (self.LbpFl[4] - self.LbpFl[0])/(0.05*0.01)

        EurocRho = (self.Euroc[4] - self.Euroc[0])/(0.05*0.01)
        EuropRho = (self.Europ[4] - self.Europ[0])/(0.05*0.01)

        return [AfixRho, AfloatRho, LbcRho, LbpRho, LbcFlRho, LbpFlRho, EurocRho, EuropRho]
        

In [98]:
greeks = Greeks()
print(greeks.delta())
print(greeks.theta())
print(greeks.vega())
print(greeks.rho())

[0.6033040000000005, 0.058715000000000295, 1.1341370000000026, -0.7756690000000006, 0.16644999999999754, 0.13386099999999956, 0.6478329999999985, -0.3524440000000002]
[-3.3725159999998127, -3.5562239999999754, -10.5210000000001, -4.12272000000015, -8.879220000000032, -5.7645000000002184, -6.404580000000159, -1.648331999999865]
[21.55649999999998, 22.05799999999991, 82.21849999999975, 65.0725000000003, 65.27949999999905, 82.01199999999974, 37.45049999999939, 37.2435000000002]
[24.30600000000105, 26.987999999999346, 45.940000000001646, -47.860000000000014, 46.824000000000865, -48.74200000000073, 53.15999999999832, -41.52199999999873]


In [100]:
deltaformatted = [f"{val:0.6f}" for val in greeks.delta()]
thetaformatted = [f"{val:0.6f}" for val in greeks.theta()]
vegaformatted = [f"{val:0.6f}" for val in greeks.vega()]
rhoformatted = [f"{val:0.6f}" for val in greeks.rho()]

print(deltaformatted)
print(thetaformatted)
print(vegaformatted)
print(rhoformatted)

['0.603304', '0.058715', '1.134137', '-0.775669', '0.166450', '0.133861', '0.647833', '-0.352444']
['-3.372516', '-3.556224', '-10.521000', '-4.122720', '-8.879220', '-5.764500', '-6.404580', '-1.648332']
['21.556500', '22.058000', '82.218500', '65.072500', '65.279500', '82.012000', '37.450500', '37.243500']
['24.306000', '26.988000', '45.940000', '-47.860000', '46.824000', '-48.742000', '53.160000', '-41.522000']


In [65]:
optionType = ['Asian Call, fixed strike price','Asian Call, floating strike price ','Lookback Call, maximum, fixed strike price','Lookback Put, minimum, fixed strike price','Lookback Call, minimum, floating strike price','Lookback Put, maximum, floating strike price']

In [67]:
header = ['Option Type', 'Option Price', 'Delta','Theta','Vega', 'Rho']
table = [
    [optionType[0], Afix00, deltaformatted[0], thetaformatted[0], vegaformatted[0],rhoformatted[0]],
    [optionType[1], Afloat00, deltaformatted[1], thetaformatted[1], vegaformatted[1],rhoformatted[1]],
    [optionType[2], Lbc00, deltaformatted[2], thetaformatted[2], vegaformatted[2],rhoformatted[2]],
    [optionType[3], Lbp00, deltaformatted[3], thetaformatted[3], vegaformatted[3],rhoformatted[3]],
    [optionType[4], LbcFl00, deltaformatted[4], thetaformatted[4], vegaformatted[4],rhoformatted[4]],
    [optionType[5], LbpFl00, deltaformatted[5], thetaformatted[5], vegaformatted[5],rhoformatted[5]],
]

In [69]:
print(tabulate(table,header))

Option Type                                      Option Price      Delta      Theta     Vega      Rho
---------------------------------------------  --------------  ---------  ---------  -------  -------
Asian Call, fixed strike price                        5.72505   0.603304   -3.37252  21.5565   24.306
Asian Call, floating strike price                     5.87149   0.058715   -3.55622  22.058    26.988
Lookback Call, maximum, fixed strike price           18.2908    1.13414   -10.521    82.2185   45.94
Lookback Put, minimum, fixed strike price            11.7403   -0.775669   -4.12272  65.0725  -47.86
Lookback Call, minimum, floating strike price        16.645     0.16645    -8.87922  65.2795   46.824
Lookback Put, maximum, floating strike price         13.3861    0.133861   -5.7645   82.012   -48.742


### Results and Comparisons

Option prices based on initial conditions, and Greeks for every option type are listed as follow: 

| Option Type | Option Price | Delta | Theta | Vega | Rho |
|--------------|--------------|-------|-------|------|-----|
| Asian Call option, fixed strike price | 5.725051 | 0.603304 | -3.372516 | 21.556500 | 24.306000 |
| Asian Call option, floating strike price | 5.871493 | 0.058715 | -3.556224 | 22.058000 | 26.988000 |
| Lookback Call, maximum, fixed strike price | 18.290788 | 1.134137 | -10.521000 | 82.218500 | 45.940000 |
| Lookback Put, minimum, fixed strike price | 11.740252 | -0.775669 | -4.122720 | 65.072500 | -47.860000 |
| Lookback Call, minimum, floating strike price | 16.644977 | 0.166450 | -8.879220 | 65.279500 | 46.824000 |
| Lookback Put, maximum, floating strike price | 13.386062 | 0.133861 | -5.764500 | 82.012000 | -48.742000 |

**Comparisons:**
Calculate the option price of European options (call and put), and Greeks respectively, as the benchmarks. Compare the Greeks of other options with European options.

In [30]:
print(f"European Call Option: {option1.EuroOption[0]:0.6f}")
print(f"European Put Option: {option1.EuroOption[1]:0.6f}")

European Call Option: 10.435249
European Put Option: 5.530524


In [36]:
print(f"European Call Option: {optionS1.EuroOption[0]:0.6f}")
print(f"European Put Option: {optionS1.EuroOption[1]:0.6f}")

European Call Option: 11.083082
European Put Option: 5.178080


In [40]:
print(f"European Call Option: {optionS2.EuroOption[0]:0.6f}")
print(f"European Put Option: {optionS2.EuroOption[1]:0.6f}")

European Call Option: 10.409834
European Put Option: 5.523983


In [44]:
print(f"European Call Option: {optionS3.EuroOption[0]:0.6f}")
print(f"European Put Option: {optionS3.EuroOption[1]:0.6f}")

European Call Option: 10.510150
European Put Option: 5.605011


In [48]:
print(f"European Call Option: {optionS4.EuroOption[0]:0.6f}")
print(f"European Put Option: {optionS4.EuroOption[1]:0.6f}")

European Call Option: 10.461829
European Put Option: 5.509763


In [50]:
Euroc00 = round(option1.EuroOption[0],6)
Europ00 = round(option1.EuroOption[1],6)
Euroc01 = round(optionS1.EuroOption[0],6)
Europ01 = round(optionS1.EuroOption[1],6)
Euroc02 = round(optionS2.EuroOption[0],6)
Europ02 = round(optionS2.EuroOption[1],6)
Euroc03 = round(optionS3.EuroOption[0],6)
Europ03 = round(optionS3.EuroOption[1],6)
Euroc04 = round(optionS4.EuroOption[0],6)
Europ04 = round(optionS4.EuroOption[1],6)

In [54]:
print(Euroc00 ,Euroc01 ,Euroc02 ,Euroc03 ,Euroc04 )
print(Europ00 ,Europ01 ,Europ02 ,Europ03 ,Europ04 )

10.435249 11.083082 10.409834 10.51015 10.461829
5.530524 5.17808 5.523983 5.605011 5.509763


In [76]:
EuroOption  = ['European Call','European Put']

In [102]:
header = ['Option Type', 'Option Price', 'Delta','Theta','Vega', 'Rho']
table = [
    [EuroOption[0], Euroc00, deltaformatted[6], thetaformatted[6], vegaformatted[6],rhoformatted[6]],
    [EuroOption[1], Europ00, deltaformatted[7], thetaformatted[7], vegaformatted[7],rhoformatted[7]],
]

In [104]:
print(tabulate(table,header))

Option Type      Option Price      Delta     Theta     Vega      Rho
-------------  --------------  ---------  --------  -------  -------
European Call        10.4352    0.647833  -6.40458  37.4505   53.16
European Put          5.53052  -0.352444  -1.64833  37.2435  -41.522


**The benchmark, the prices and Greeks of plain European option are listed below:**
| Option Type | Option Price | Delta  | Theta  | Vega  | Rho   |
|-------------|--------------|--------|--------|-------|-------|
| European Call option | 10.435249 | 0.647833 | -6.404580 | 37.450500 | 53.160000 |
| European Put option | 5.530524  | -0.352444 | -1.648332 | 37.243500 | -41.522000 |


### Observations and Discussion
**Asian Call option**

| Option Type | Option Price | Delta  | Theta  | Vega  | Rho   |
|-------------|--------------|--------|--------|-------|-------|
| European Call option | 10.435249 | 0.647833 | -6.404580 | 37.450500 | 53.160000 |
| Asian Call option, fixed strike price | 5.725051 | 0.603304 | -3.372516 | 21.556500 | 24.306000 |
| Asian Call option, floating strike price | 5.871493 | 0.058715 | -3.556224 | 22.058000 | 26.988000 |

-Asian call options are less expensive, since their averaging mechanics make them typically exhibiting lower volatility, compared to standard options.

-For greeks, absolute values are all less than those of standard European options.The averaging feature in Asian options leads to a smoothing effect on all Greeks, making them generally smaller in magnitude compared to standard options.

-Delta: less than European option. Averaging effect smooths out price fluctuations, and reduces its sensitivity  to immediate changes in the underlying asset price.Delta of option with floating strike price is far less.

-Theta: it measures time decay. Theta in magnitude of Asian options are lower compared to European option.

-Vega: less that European option. the volatility of the average price over time is less than that of the spot price, which result in lower Vega.

-Rho:  smaller than that of European options.

**Lookback Call option**

| Option Type | Option Price | Delta  | Theta  | Vega  | Rho   |
|-------------|--------------|--------|--------|-------|-------|
| European Call option | 10.435249 | 0.647833 | -6.404580 | 37.450500 | 53.160000 |
| Lookback Call, maximum, fixed strike price | 18.290788 | 1.134137 | -10.521000 | 82.218500 | 45.940000 |
| Lookback Call, minimum, floating strike price | 16.644977 | 0.166450 | -8.879220 | 65.279500 | 46.824000 |

-Lookback options offer holders the most favorable price , so they are worth considerably more than European options.

-Delta: delta of lookback call options can be greater than or less than that of European option. My results show that the one with fixed strike price is larger than European option, and the one with floating strike price is less.

-Theta: larger than European option in magnitude.

-Vega: larger than European option, indicating that lookback call options are more sensitive to volatility changes.

-Rho: less that European option.

**Lookback Put option**

| Option Type | Option Price | Delta  | Theta  | Vega  | Rho   |
|-------------|--------------|--------|--------|-------|-------|
| European Put option | 5.530524  | -0.352444 | -0.006541 | 0.074487 | -0.020761 |
| Lookback Put, minimum, fixed strike price | 11.740252 | -0.775669 | -4.122720 | 65.072500 | -47.860000 |
| Lookback Put, maximum, floating strike price | 13.386062 | 0.133861 | -1.648332 | 37.243500 | -41.522000 |

-Lookback options offer high payoffs for puts potentially, so they are worth considerably more than European options.

-Delta: the option with fixed strike price is larger than European option in magnitude, whereas the option with floating striking price has opposite changing direction to that of European option, and is less in magnitude.
Generally, put options have negative delta. However, for lookback put options, the trader has rights to sell at the highest, which means put premiums rise when the underlying asset rises.

-Theta: larger than European option in magnitude.

-Vega: larger than European option.

-Rho: larger than European option in magnitude.

**Summary**

-Asian options show smoothing effects, resulting from their averaging features.

-Lookback options show more sensitive to changes in underlying parameters than standard European options, reflected in larger magnitudes for more Greeks.

-Theta is always negative, since time moves in the same direction. When an option is purchased, the value of the option begins to diminish until it expires.

-Vega of all options are positive.

-Call options have positive Rho, whereas put options have negative Rho.

### Future Work

-Calculate and discuss Greeks under different time periods or situations. Possible scenarios: selecting various time spots within the contract period, starting points, far from expirations, intermediate periods, and closer to expirations, and then calculating Greeks respectively to discover the trends. Or do comparisons of Greeks between in-the-money and out-of-money. Also, consider how to use sensitivities in trading strategies.

-Use other methods to do calculations and comparisons, such as deriving modified BS model adapted to exotic options and then use finite difference method to solve the PDE. Although, some research derived the closed-form formula and approximated formula (Zhang, 1998, and Haug, 2007), it is not easy to achieve the calculations.

### Reference
1. Hull, John. Options, Futures, and Other Derivatives. Ninth edition. Boston: Pearson, 2015.
2. Haug, Espen Gaarder. The Complete Guide to Option Pricing Formulas. 2. ed. New York, NY: McGraw-Hill, 2007.
3. Paul Glasserman. Monte Carlo Methods in Financial Engineering. Springer-Verlag New York, 2004
4. Paul Wilmott. Paul Wilmott introduces Quantitative Finance. Wiley, 2007
5. Peter G. Zhang. Exotic Options A Guide to Second Generation Options. World Scientific, 1998